In [ ]:
import datasets
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# Load dataset
data = datasets.load_dataset('path_to_your_dataset')  # Ensure your dataset is formatted correctly

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('model_name_or_path')
model = AutoModelForCausalLM.from_pretrained('model_name_or_path')

# Add adapter configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM"
)

# Get the PEFT model
model = get_peft_model(model, peft_config)

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_data = data.map(tokenize_function, batched=True)

# Prepare data collator
data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=False,  # Set to False for CPU training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained('path_to_save_your_model')
tokenizer.save_pretrained('path_to_save_your_model')


In [ ]:
import datasets
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model

# Load dataset
data = datasets.load_dataset('path_to_your_dataset')  # Ensure your dataset is formatted correctly

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('model_name_or_path')
model = AutoModelForCausalLM.from_pretrained('model_name_or_path')

# Add adapter configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM"
)

# Get the PEFT model
model = get_peft_model(model, peft_config)

# Tokenize the dataset
def tokenize_function(examples):
    inputs = [ex['user_input'] for ex in examples]
    responses = [ex['assistant_response'] for ex in examples]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True)
    model_inputs["labels"] = tokenizer(responses, padding="max_length", truncation=True)["input_ids"]
    return model_inputs

tokenized_data = data.map(tokenize_function, batched=True)

# Prepare data collator
data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
    fp16=False,  # Set to False for CPU training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
)

# Train the model
trainer.train()

# Save the model and tokenizer
model.save_pretrained('path_to_save_your_model')
tokenizer.save_pretrained('path_to_save_your_model')
